In [7]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

#device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data',train=True,transform = transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root='../../data',train=False,transform = transforms.ToTensor())

# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        # Move tensors to configured device
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        
        # Forward Pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
# Test the model
# Don't need to compute the gradients

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Accuracy of the network on the 10000 test images : {} %'.format(100*correct/total))
    
# Save the model checkpoint
torch.save(model.state_dict(),'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.4374
Epoch [1/5], Step [200/600], Loss: 0.4113
Epoch [1/5], Step [300/600], Loss: 0.2544
Epoch [1/5], Step [400/600], Loss: 0.1704
Epoch [1/5], Step [500/600], Loss: 0.1305
Epoch [1/5], Step [600/600], Loss: 0.1317
Epoch [2/5], Step [100/600], Loss: 0.1715
Epoch [2/5], Step [200/600], Loss: 0.0988
Epoch [2/5], Step [300/600], Loss: 0.0597
Epoch [2/5], Step [400/600], Loss: 0.0584
Epoch [2/5], Step [500/600], Loss: 0.1539
Epoch [2/5], Step [600/600], Loss: 0.0501
Epoch [3/5], Step [100/600], Loss: 0.0766
Epoch [3/5], Step [200/600], Loss: 0.0656
Epoch [3/5], Step [300/600], Loss: 0.0447
Epoch [3/5], Step [400/600], Loss: 0.0737
Epoch [3/5], Step [500/600], Loss: 0.0514
Epoch [3/5], Step [600/600], Loss: 0.1296
Epoch [4/5], Step [100/600], Loss: 0.0749
Epoch [4/5], Step [200/600], Loss: 0.0955
Epoch [4/5], Step [300/600], Loss: 0.0628
Epoch [4/5], Step [400/600], Loss: 0.0338
Epoch [4/5], Step [500/600], Loss: 0.0183
Epoch [4/5], Step [600/600], Loss: